[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/fac2003/repro_deepspeed_multi_modality_perceiver_stage_3/blob/main/DeepSpeedMultiModalityPerceiver.ipynb)

In [ ]:
! pip install perceiver-multi-modality-pytorch==1.1.0 
# !  pip install deepspeed==0.3.15

In [ ]:
! git clone https://github.com/microsoft/DeepSpeed.git

Cloning into 'DeepSpeed'...
remote: Enumerating objects: 8510, done.
remote: Counting objects: 100% (965/965), done.
remote: Compressing objects: 100% (462/462), done.
remote: Total 8510 (delta 645), reused 764 (delta 491), pack-reused 7545
Receiving objects: 100% (8510/8510), 17.08 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (5730/5730), done.


In [ ]:
! cd DeepSpeed; pip install .

Processing /content/DeepSpeed
  Created wheel for deepspeed: filename=deepspeed-0.3.15+03d24fe-cp37-none-any.whl size=409651 sha256=d4ac9df61c062f446750441da554bb0ee7d103c4fc607574fd8591dd1e280423
  Stored in directory: /tmp/pip-ephem-wheel-cache-ka8tovsl/wheels/ce/45/03/3ff604290a121d63cfe2874fc4f4f36ba7053e7d83932decd8
Successfully built deepspeed
  Found existing installation: deepspeed 0.3.15+03d24fe
    Uninstalling deepspeed-0.3.15+03d24fe:
      Successfully uninstalled deepspeed-0.3.15+03d24fe


In [ ]:
!pip install mpi4py
!!pip install torch==1.7.1+cu110  -f https://download.pytorch.org/whl/torch_stable.html


['Looking in links: https://download.pytorch.org/whl/torch_stable.html',
 'Requirement already satisfied: torch==1.7.1+cu110 in /usr/local/lib/python3.7/dist-packages (1.7.1+cu110)',
 'Requirement already satisfied: typing-extensions in /usr/local/lib/python3.7/dist-packages (from torch==1.7.1+cu110) (3.7.4.3)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.7/dist-packages (from torch==1.7.1+cu110) (1.19.5)']

In [ ]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [NO] ....... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
 [WARNING]  sparse_attn requires one of the following commands '['llvm-config', 'llvm-config-9']', but it does not exist!
 [WARNING]  sparse_attn requires CUDA version 10.1+, does not currently support >=11 or <10.1
sparse_attn ............ [NO] ....... [NO]
transformer

In [ ]:
num_epochs=10000

from perceiver_pytorch.multi_modality_perceiver import  InputModality
from perceiver_pytorch.multi_modality_with_text_perceiver import MultiModalityWithTextPerceiver, InputModalityWithEmbedding
import torch

import deepspeed

video_modality = InputModalityWithEmbedding(
    name='video',
    input_channels=3,  # number of channels for each token of the input
    input_axis=3,  # number of axes, 3 for video)
    num_freq_bands=6,  # number of freq bands, with original value (2 * K + 1)
    max_freq=4.,  # maximum frequency, hyperparameter depending on how fine the data is
)
image_modality = InputModalityWithEmbedding(
    name='image',
    input_channels=3,  # number of channels for each token of the input
    input_axis=2,  # number of axes, 2 for images
    num_freq_bands=6,  # number of freq bands, with original value (2 * K + 1)
    max_freq=4.,  # maximum frequency, hyperparameter depending on how fine the data is
)
audio_modality = InputModalityWithEmbedding(
    name='audio',
    input_channels=1,  # number of channels for mono audio
    input_axis=1,  # number of axes, 2 for images
    num_freq_bands=6,  # number of freq bands, with original value (2 * K + 1)
    max_freq=8.,  # maximum frequency, hyperparameter depending on how fine the data is
)
with deepspeed.zero.Init(
):
 model = MultiModalityWithTextPerceiver(
    modalities=(video_modality, image_modality),
    depth=2,  # depth of net, combined with num_latent_blocks_per_layer to produce full Perceiver
    num_latents=12,
    # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim=64,  # latent dimension
    cross_heads=1,  # number of heads for cross attention. paper said 1
    latent_heads=2,  # number of heads for latent self attention, 8
    cross_dim_head=64,
    latent_dim_head=64,
    num_classes=10,  # output number of classes
    attn_dropout=0.,
    ff_dropout=0.,
    weight_tie_layers=True,
    num_latent_blocks_per_layer=2 # Note that this parameter is 1 in the original Lucidrain implementation
    # whether to weight tie layers (optional, as indicated in the diagram)
 )


ds_config={    "train_batch_size": 3,
    "steps_per_print": 2000,
    "optimizer": {
      "type": "Adam",
      "params": {
        "lr": 0.001,
        "betas": [
          0.8,
          0.999
        ],
        "eps": 1e-8,
        "weight_decay": 3e-7
      }
    },
    "fp16": {
      "enabled": True,
      "loss_scale": 1,
      "initial_scale_power": 32,
      "loss_scale_window": 1000,
      "hysteresis": 2,
      "min_loss_scale": 1
    },
    "scheduler": {
      "type": "WarmupLR",
      "params": {
        "warmup_min_lr": 0,
        "warmup_max_lr": 0.001,
        "warmup_num_steps": 1000
      }
    },
    "wall_clock_breakdown": False

  }
stage_3=True
if stage_3:
  ds_config.update({
      "zero_optimization": {
    "stage": 3,
    "cpu_offload": False,
    "cpu_offload_params": False,
    "overlap_comm": False,
    "contiguous_gradients": False,
    "stage3_max_live_parameters": 600000,
    "stage3_max_reuse_distance": 10000000,
    "stage3_prefetch_bucket_size": 20000,
    "stage3_param_persistence_threshold": 10000,
    "reduce_bucket_size": 300000,
    "sub_group_size": 1e6
  }})
model=model.to(torch.device('cuda'))
parameters = filter(lambda p: p.requires_grad, model.parameters())
# Initialize DeepSpeed to use the following features
# 1) Distributed model
# 2) Distributed data loader
# 3) DeepSpeed optimizer
model_engine, optimizer, trainloader, __ = deepspeed.initialize( model=model,
                                                                model_parameters=parameters,
                                                                config_params=ds_config
)

for epoch in range(num_epochs):  # loop over the dataset multiple times

      running_loss = 0.0

      image_inputs= torch.rand(size=(3, 64, 64, 3), requires_grad=True).to(model_engine.local_rank)
      video_inputs= torch.rand(size=(3, 2, 64, 64, 3), requires_grad=True).to(model_engine.local_rank)
      with torch.cuda.amp.autocast():
        outputs = model_engine({
            'image': image_inputs,
            'video': video_inputs
            }
        )
      
      loss = outputs.mean()

      model_engine.backward(loss)
      model_engine.step()
print("DONE")

[2021-04-24 16:16:23,518] [INFO] [distributed.py:37:init_distributed] Not using the DeepSpeed or torch.distributed launchers, attempting to detect MPI environment...
[2021-04-24 16:16:23,965] [INFO] [distributed.py:89:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.28.0.2, master_port=29500
[2021-04-24 16:16:23,966] [INFO] [distributed.py:47:init_distributed] Initializing torch distributed with backend: nccl
nn.functional.linear has been overridden with a more memory efficient version. This will persist unless manually reset.
[2021-04-24 16:16:27,160] [INFO] [logging.py:60:log_dist] [Rank 0] DeepSpeed info: version=0.3.15+03d24fe, git-hash=03d24fe, git-branch=master
[2021-04-24 16:16:27,162] [WARNING] [config.py:78:_sanity_check] DeepSpeedConfig: cpu_offload is deprecated. Please use offload_optimizer.
[2021-04-24 16:16:27,163] [WARNING] [config.py:78:_sanity_check] DeepSpeedConfig: cpu_offload_params is deprecated. Please use offload

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:354: FutureWarning: torch.cuda.max_memory_cached has been renamed to torch.cuda.max_memory_reserved
  FutureWarning)


Emitting ninja build file /root/.cache/torch_extensions/utils/build.ninja...
Building extension module utils...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module utils...
Time to load utils op: 12.045355319976807 seconds
[2021-04-24 16:17:01,878] [INFO] [utils.py:583:see_memory_usage] Before creating fp16 partitions
[2021-04-24 16:17:01,880] [INFO] [utils.py:588:see_memory_usage] MA 0.0 GB         Max_MA 0.0 GB         CA 0.0 GB         Max_CA 0 GB 
[2021-04-24 16:17:01,883] [INFO] [utils.py:593:see_memory_usage] CPU Virtual Memory:  used = 2.66 GB, percent = 20.9%
[2021-04-24 16:17:01,885] [INFO] [stage3.py:39:print_rank_0] fp16 group 0 has 1 subgroups
[2021-04-24 16:17:01,901] [INFO] [stage3.py:39:print_rank_0] Swappable FP32 Partitions: count=0 size= 0.00 GB
[2021-04-24 16:17:01,902] [INFO] [stage3.py:39:print_rank_0] In-Memory FP32 Partitions: count=1 size= 0.00 GB
[2021-04-24 16:17:01,905] [IN

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:354: FutureWarning: torch.cuda.max_memory_cached has been renamed to torch.cuda.max_memory_reserved
  FutureWarning)


[2021-04-24 16:17:02,097] [INFO] [config.py:747:print]   zero_allow_untested_optimizer  False
[2021-04-24 16:17:02,099] [INFO] [config.py:747:print]   zero_config .................. {
    "stage": 3, 
    "contiguous_gradients": false, 
    "reduce_scatter": false, 
    "reduce_bucket_size": 3.000000e+05, 
    "allgather_partitions": true, 
    "allgather_bucket_size": 5.000000e+08, 
    "overlap_comm": false, 
    "load_from_fp32_weights": true, 
    "elastic_checkpoint": true, 
    "offload_param": null, 
    "offload_optimizer": null, 
    "sub_group_size": 1.000000e+06, 
    "prefetch_bucket_size": 2.000000e+04, 
    "param_persistence_threshold": 1.000000e+04, 
    "max_live_parameters": 6.000000e+05, 
    "max_reuse_distance": 1.000000e+07, 
    "gather_fp16_weights_on_model_save": false
}
[2021-04-24 16:17:02,100] [INFO] [config.py:747:print]   zero_enabled ................. True
[2021-04-24 16:17:02,101] [INFO] [config.py:747:print]   zero_optimization_stage ...... 3
[2021-04-2

RuntimeError: ignored

In [ ]:
ONE